In [1]:
import sys
sys.path.append('/Users/apple/Desktop/AiCore Bootcamp/multinational-retail-data-centralisation939')

In [2]:
from src.database_utils import DatabaseConnector
from src.data_cleaning import DataCleaning
from src.data_extraction import DataExtractor
from src.data_extraction import header


In [3]:
import numpy as np
import pandas as pd

## Task 3: Extract and clean user data

In [ ]:
# Step 2: Read the credentials yaml file and return a dictionary of the credentials.

instance = DatabaseConnector('../db_creds.yaml')
db_creds = instance.read_db_creds()

In [ ]:
# Step 3: Read the credentials from the return of read_db_creds and initialise and return an sqlalchemy database engine.
db_engine = instance.init_db_engine()

In [ ]:
# Step 4: Create a method list_db_tables to list all the tables in the database so you know which tables you can extract data from.
table_names = instance.list_db_tables()
print(table_names)

In [ ]:
# Step 5:
# Develop a method called read_rds_table in your DataExtractor class which will extract the database table to a pandas DataFrame.

extractor = DataExtractor()
user_data = extractor.read_rds_table('legacy_users')
user_data.head()

In [ ]:
# Step 6: Clean data
cleaner = DataCleaning()
try:
    cleaned_user_data = cleaner.clean_user_data(user_data)
except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
cleaned_user_data.head()

In [ ]:
# Step 7+8: Upload to SQL
instance = DatabaseConnector('../sales_data_creds.yaml')
instance.upload_to_db(cleaned_user_data, 'dim_users')

## Task 4: Extracting users and cleaning card details

In [ ]:
# Step 2:
# Create a method in your DataExtractor class called retrieve_pdf_data, which takes in a link as an argument and returns a pandas DataFrame.

extractor = DataExtractor()
pdf_data = extractor.retrieve_pdf_data('https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf')

In [ ]:
pdf_data.head()

In [ ]:
#  Step 3: Clean data
cleaner = DataCleaning()
cleaned_card_data = cleaner.clean_card_data(pdf_data)

In [ ]:
cleaned_card_data.head()

In [ ]:
# Step 4: Upload to SQL
instance = DatabaseConnector('../sales_data_creds.yaml')
instance.upload_to_db(cleaned_card_data, 'dim_card_details')

## Task 5: Extract and clean details of the store

In [4]:
# Step 1+2:
# Create a method in your DataExtractor class called list_number_of_stores which returns the number of stores to extract. It should take in the number of stores endpoint and header dictionary as an argument.

number_of_stores_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores'
extractor = DataExtractor()
number_of_stores = extractor.list_number_of_stores(number_of_stores_endpoint,header)

Number of stores: 451


In [5]:
# Step 3:
# Create another method retrieve_stores_data which will take the retrieve a store endpoint as an argument and extracts all the stores from the API saving them in a pandas DataFrame.
store_endpoint = 'https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/'

store_data = extractor.retrieve_stores_data(store_endpoint, number_of_stores, header)

In [6]:
# Step 4: Clean data
cleaner = DataCleaning()
cleaned_store_data = cleaner.clean_store_data(store_data)

In [7]:
cleaned_store_data.head()

,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,NaN,<NA>,N/A,<NA>,325,2010-06-12,Web Portal,<NA>,GB,Europe
1,"Flat 72W\nSally isle\nEast Deantown\nE7B 8EB, ...",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,"Heckerstraße 4/5\n50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,"5 Harrison tunnel\nSouth Lydia\nWC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,Studio 6\nStephen landing\nSouth Simon\nB77 2W...,53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe


In [8]:
# Step 5: Upload to SQL
instance = DatabaseConnector('../sales_data_creds.yaml')
instance.upload_to_db(cleaned_store_data,'dim_store_details')

451

In [12]:
max_length_index = cleaned_store_data['continent'].str.len().idxmax()

# Retrieve the entry with the longest string length
entry_with_max_length = cleaned_store_data.loc[max_length_index, 'continent']

print("Entry with the longest length:", len(entry_with_max_length))

Entry with the longest length: 7


## Task 6: Extract and clean the product details

In [14]:
# Step 1: Extract data from s3 bucket into pd dataframe
extractor = DataExtractor()
products_data = extractor.extract_from_s3('s3://data-handling-public/products.csv')

In [15]:
products_data.head()

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_avaliable,R7-3126933h
1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,487128731892,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_avaliable,C2-7287916l
2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_avaliable,S7-1175877v
3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n
4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_avaliable,B6-2596063a


In [ ]:
# Step 2: Convert product weights to correct units.
# Method is private as is used in the clean_products_data function.
# See data_cleaning.py convert_product_weights.

In [16]:
# Step 3: Clean whole dataframe
cleaner = DataCleaning()
cleaned_products_data = cleaner.clean_products_data(products_data)
cleaned_products_data.head()

,product_name,product_price,weight,category,EAN,date_added,uuid,removed,product_code,new_weight
0,FurReal Dazzlin' Dimples My Playful Dolphin,£39.99,1.6kg,toys-and-games,7425710935115,2005-12-02,83dc0a69-f96f-4c34-bcb7-928acae19a94,Still_available,R7-3126933h,<NA>
1,Tiffany's World Day Out At The Park,£12.99,0.48kg,toys-and-games,<NA>,2006-01-09,712254d7-aea7-4310-aff8-8bcdd0aec7ff,Still_available,C2-7287916l,0.48
2,Tiffany's World Pups Picnic Playset,£7.00,590g,toys-and-games,1945816904649,1997-03-29,b089ef6f-b628-4e37-811d-fffe0102ba64,Still_available,S7-1175877v,0.59
3,Tiffany's World Wildlife Park Adventures,£12.99,540g,toys-and-games,1569790890899,2013-03-20,d55de422-8b98-47d6-9991-e4bc4c5c0cb0,Removed,D8-8421505n,0.54
4,Cosatto Cosy Dolls Pram,£30.00,1.91kg,toys-and-games,7142740213920,2007-12-23,7945b657-cb02-4cc5-96cf-f65ed0a8f235,Still_available,B6-2596063a,1.91


In [12]:
cleaned_products_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1772 entries, 2 to 1481
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1482 non-null   string
 1   product_price  1482 non-null   string
 2   weight         1482 non-null   object
 3   category       1482 non-null   string
 4   EAN            1482 non-null   string
 5   date_added     1482 non-null   string
 6   uuid           1482 non-null   string
 7   removed        1482 non-null   string
 8   product_code   1482 non-null   string
 9   new_weight     1482 non-null   string
dtypes: object(1), string(9)
memory usage: 216.8+ KB


In [18]:
# Step 4: Upload to SQL
instance = DatabaseConnector('../sales_data_creds.yaml')
instance.upload_to_db(cleaned_products_data,'dim_products')

853

In [10]:
print(cleaned_products_data[cleaned_products_data['weight'] == '440g'])


Empty DataFrame
Columns: [product_name, product_price, weight, category, EAN, date_added, uuid, removed, product_code]
Index: []


## Task 7: Retrieve and clean the orders table.

In [ ]:
# Step 1: Find table containing information on product orders.
connector = DatabaseConnector('../db_creds.yaml')
connector.list_db_tables()

In [ ]:
# Step 2: Read orders_table file
extractor = DataExtractor()
orders_table_data = extractor.read_rds_table('orders_table')

In [ ]:
orders_table_data.head()

In [ ]:
# Step 3: Clean orders_data
cleaned_orders_data = DataCleaning().clean_orders_data(orders_table_data)

In [ ]:
cleaned_orders_data.head()

In [ ]:
cleaned_orders_data.info()

In [ ]:
# Step 4: Upload to SQL
connector = DatabaseConnector('../sales_data_creds.yaml')
connector.upload_to_db(cleaned_orders_data, 'orders_table')

## Task 8: Retrieve and clean the date events data.

In [4]:
# Step 1: Extract JSON file containing date events data
path = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/date_details.json'

date_events_data = DataExtractor().extract_from_json(path)

In [5]:
date_events_data.head()

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44


In [6]:
# Step 2: Clean data
cleaned_date_events_data = DataCleaning().clean_date_data(date_events_data)

In [7]:
cleaned_date_events_data.head()

,timestamp,month,year,day,time_period,date_uuid
0,22:00:06,9,2012,19,Evening,3b7ca996-37f9-433f-b6d0-ce8391b615ad
1,22:44:06,2,1997,10,Evening,adc86836-6c35-49ca-bb0d-65b6507a00fa
2,10:05:37,4,1994,15,Morning,5ff791bf-d8e0-4f86-8ceb-c7b60bef9b31
3,17:29:27,11,2001,6,Midday,1b01fcef-5ab9-404c-b0d4-1e75a0bd19d8
4,22:40:33,12,2015,31,Evening,dfa907c1-f6c5-40f0-aa0d-40ed77ac5a44


In [9]:
# Step 3: Upload to SQL
connector = DatabaseConnector('../sales_data_creds.yaml')
connector.upload_to_db(cleaned_date_events_data, 'dim_date_times')

147

In [8]:
cleaned_date_events_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 120147 entries, 0 to 120160
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   timestamp    120123 non-null  object
 1   month        120123 non-null  string
 2   year         120123 non-null  string
 3   day          120123 non-null  string
 4   time_period  120123 non-null  string
 5   date_uuid    120123 non-null  string
dtypes: object(1), string(5)
memory usage: 6.4+ MB
